In [1]:
!pip install ultralytics

In [2]:
from ultralytics import YOLO
from IPython.display import Image

In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="KQy5UhhJbKooQ1UHTO9R")
project = rf.workspace("brac-university-8frgk").project("license-plate-detection-lluwr")
version = project.version(1)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...


In [4]:
!cat /kaggle/working/license-plate-detection-1/data.yaml


train: ../train/images
val: ../valid/images
test: ../test/images

nc: 27
names: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'ba', 'ca', 'chattogram', 'da', 'dhaka', 'ga', 'gha', 'ha', 'ja', 'jashore', 'ka', 'kha', 'khulna', 'metro', 'na', 'ta', 'va']

roboflow:
  workspace: brac-university-8frgk
  project: license-plate-detection-lluwr
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/brac-university-8frgk/license-plate-detection-lluwr/dataset/1

In [5]:
!nvidia-smi

Sun Aug 24 10:15:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P0             28W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
!pip install PyMuPDF==1.23.1

In [7]:

!pip uninstall paddleocr paddlepaddle paddlepaddle-gpu -y
!pip install paddlepaddle==2.6.2
!pip install paddleocr

Found existing installation: paddleocr 3.2.0
Uninstalling paddleocr-3.2.0:
  Successfully uninstalled paddleocr-3.2.0
Found existing installation: paddlepaddle 2.6.2
Uninstalling paddlepaddle-2.6.2:
  Successfully uninstalled paddlepaddle-2.6.2
  Using cached paddlepaddle-2.6.2-cp311-cp311-manylinux1_x86_64.whl.metadata (8.6 kB)
Using cached paddlepaddle-2.6.2-cp311-cp311-manylinux1_x86_64.whl (126.0 MB)
  Using cached paddleocr-3.2.0-py3-none-any.whl.metadata (29 kB)
Using cached paddleocr-3.2.0-py3-none-any.whl (75 kB)


In [9]:

import cv2
import os
import easyocr
import pytesseract
# from paddleocr import PaddleOCR
import csv
import re

model = YOLO("/kaggle/input/yolo11-best-model/pytorch/default/1/best.pt")

reader_easy = easyocr.Reader(['en'])
# ocr_paddle = PaddleOCR(use_textline_orientation=True, lang='en')

pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'  # adjust path

def ocr_easy(crop):
    result = reader_easy.readtext(crop)
    return result[0][-2].strip() if len(result) > 0 else ""

def ocr_tess(crop):
    gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    txt = pytesseract.image_to_string(
        thresh,
        config='--psm 8 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
    )
    return txt.strip()

# def ocr_padd(crop):
#     result = ocr_paddle.ocr(crop, cls=True)
#     return result[0][1][0] if len(result) > 0 else ""

def clean_plate(txt):
    return re.sub(r'[^A-Z0-9]', '', txt.upper())

ocr_engines = {
    "EasyOCR": ocr_easy,
    "Tesseract": ocr_tess,
    # "PaddleOCR": ocr_padd
}


VIDEO_DIR = "/kaggle/input/test-videos-yolov11-detections"
OUTPUT_DIR = "/kaggle/working/output_videos"
os.makedirs(OUTPUT_DIR, exist_ok=True)


engine_folders = {name: os.path.join(OUTPUT_DIR, name) for name in ocr_engines.keys()}
engine_folders["Best"] = os.path.join(OUTPUT_DIR, "Best")
for folder in engine_folders.values():
    os.makedirs(folder, exist_ok=True)


CSV_PATH = os.path.join(OUTPUT_DIR, "ocr_comparison.csv")
csv_file = open(CSV_PATH, mode="w", newline="", encoding="utf-8")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["video_name", "frame_number", "plate_easyocr", "plate_tesseract", "plate_paddleocr", "best_guess"])

resize_w, resize_h = 640, 360
min_gap = 15  # avoid duplicates
last_seen = {}

for video_file in os.listdir(VIDEO_DIR):
    if not video_file.lower().endswith((".mp4", ".avi", ".mov", ".mkv")):
        continue

    VIDEO_PATH = os.path.join(VIDEO_DIR, video_file)
    print(f"\nProcessing: {VIDEO_PATH}")

    cap = cv2.VideoCapture(VIDEO_PATH)
    fps = cap.get(cv2.CAP_PROP_FPS)
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create video writers for each engine + best
    writers = {}
    for name, folder in engine_folders.items():
        out_name = os.path.join(folder, f"{os.path.splitext(video_file)[0]}_{name}.mp4")
        writers[name] = cv2.VideoWriter(out_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (W, H))

    frame_idx = 0
    last_seen[video_file] = {}

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_small = cv2.resize(frame, (resize_w, resize_h))
        results = model.predict(frame_small, conf=0.25, iou=0.5, device=0, verbose=False)

        # Copy frames for each engine + best
        frames_out = {name: frame.copy() for name in writers.keys()}

        for r in results:
            for box in r.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
                scale_x, scale_y = W / resize_w, H / resize_h
                x1, x2 = int(x1 * scale_x), int(x2 * scale_x)
                y1, y2 = int(y1 * scale_y), int(y2 * scale_y)

                plate_crop = frame[y1:y2, x1:x2]
                if plate_crop.size == 0:
                    continue

                # Run all OCR engines
                plate_results = {}
                for name, func in ocr_engines.items():
                    plate_results[name] = clean_plate(func(plate_crop))

                # Best guess: first non-empty
                best_guess = next((txt for txt in plate_results.values() if txt != ""), "")

                # Avoid duplicates for best
                last_logged = last_seen[video_file].get(best_guess, -9999)
                if frame_idx - last_logged >= min_gap and best_guess != "":
                    last_seen[video_file][best_guess] = frame_idx
                    # Save to CSV
                    csv_writer.writerow([
                        video_file,
                        frame_idx,
                        plate_results["EasyOCR"],
                        plate_results["Tesseract"],
                        # plate_results["PaddleOCR"],
                        best_guess
                    ])
                    print(f"{video_file} | Frame {frame_idx} | Plate: {best_guess}")

                # Draw boxes and text for each engine
                for name in frames_out.keys():
                    text = best_guess if name == "Best" else plate_results.get(name, "")
                    if text != "":
                        cv2.rectangle(frames_out[name], (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(frames_out[name], text, (x1, y1 - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Write frames
        for name, writer in writers.items():
            writer.write(frames_out[name])

        frame_idx += 1
        if frame_idx % 100 == 0:
            print(f"   {frame_idx} frames processed...")

    cap.release()
    for writer in writers.values():
        writer.release()
    print(f"✅ Done processing {video_file}")

csv_file.close()
print(f"\n📂 All OCR results saved to: {CSV_PATH}")



Processing: /kaggle/input/test-videos-yolov11-detections/video4.mp4
   100 frames processed...
   200 frames processed...
   300 frames processed...
video4.mp4 | Frame 392 | Plate: 3
video4.mp4 | Frame 393 | Plate: ES
video4.mp4 | Frame 394 | Plate: SS
video4.mp4 | Frame 395 | Plate: 25
video4.mp4 | Frame 397 | Plate: 2L
video4.mp4 | Frame 398 | Plate: S54
video4.mp4 | Frame 399 | Plate: EB
   400 frames processed...
video4.mp4 | Frame 410 | Plate: CC
video4.mp4 | Frame 423 | Plate: BB
   500 frames processed...
video4.mp4 | Frame 500 | Plate: V
video4.mp4 | Frame 520 | Plate: F
   600 frames processed...
video4.mp4 | Frame 644 | Plate: 7
video4.mp4 | Frame 657 | Plate: N
video4.mp4 | Frame 658 | Plate: SN
video4.mp4 | Frame 661 | Plate: J
video4.mp4 | Frame 662 | Plate: I
   700 frames processed...
   800 frames processed...
   900 frames processed...
   1000 frames processed...
video4.mp4 | Frame 1064 | Plate: J
   1100 frames processed...
   1200 frames processed...
✅ Done processi